In [7]:
import numpy as np
import prepare_ml_dataset
import train_nn_model
import importlib
import pandas as pd

importlib.reload(prepare_ml_dataset)
importlib.reload(train_nn_model)

ModuleNotFoundError: No module named 'numpy'

In [8]:
energy = '51767680'
species = 'h'

In [9]:
directories, dataset_csv, data_settings = prepare_ml_dataset.initializ_var(energy, species)
y_name=data_settings["y_name"]
y_name_log=data_settings["y_name_log"]
feature_names=data_settings["feature_names"]
dL01=data_settings["dL01"]
release=data_settings["release"]
data_dir = directories["data_dir"]


NameError: name 'prepare_ml_dataset' is not defined

In [10]:
df_full = prepare_ml_dataset.read_csv_data(data_dir, release)      

index_good = prepare_ml_dataset.create_good_index(df_full, y_name , feature_names, dL01)

df_full = prepare_ml_dataset.calculate_log_for_y(df_full, y_name , y_name_log , index_good )

df_full = prepare_ml_dataset.scale_corrdinates(df_full)

df_full = prepare_ml_dataset.scale_indexes(df_full)

df_full = prepare_ml_dataset.scale_sw(df_full)

NameError: name 'prepare_ml_dataset' is not defined

In [11]:
n_history_total_days = 7
average_time = 300
n_history_total = n_history_total_days*24*60*60/average_time
m_history = int(n_history_total/24 + 1)
history_resolution = 2*3600
m_history = int(n_history_total/(history_resolution/average_time) + 1)
feature_names=[ 'scaled_symh', 'scaled_ae','scaled_asyh', 'scaled_asyd']
m_feature = len(feature_names)

index_difference = history_resolution/average_time
feature_history_names = ["" for x in range(m_feature*m_history)]
index0 = int(n_history_total)
index1 = df_full.index[-1]

df_history = data=np.zeros((index1-index0+1, len(feature_history_names)))

ihf=0
for feature_name in feature_names:
    for k in range(m_history):
        name = feature_name + '_' + str(k*2)+'h'
        feature_history_names[ihf] = name
        temp = np.array(df_full.loc[(index0 - index_difference*k):(index1-index_difference*k), feature_name])
        df_history[:,k] = temp
        # df_full.loc[index0:index1,name] = np.array(df_full.loc[(index0 - index_difference*k):(index1-index_difference*k), feature_name])  
        ihf = ihf + 1

df_full.loc[index0:index1, feature_history_names] = df_history

NameError: name 'df_full' is not defined

In [ ]:
df_full, index_good, data_settings["feature_history_names"] = prepare_ml_dataset.create_df_full('rel05', data_settings["y_name"] , data_settings["y_name_log"] ,  data_settings["feature_names"] , data_settings["dL01"] , data_settings["number_history"] , data_settings["history_resolution"] , data_settings["average_time"], data_settings["forecast"] , directories["data_dir"] )


In [ ]:
df_history = np.empty((index1-index0+1, m_feature))


In [ ]:
x_train, x_valid, x_test, y_train, y_valid, y_test  = prepare_ml_dataset.load_csv_data(dataset_csv)


In [ ]:
print(x_train.shape, x_valid.shape, x_test.shape, y_train.shape, y_valid.shape, y_test.shape)

In [ ]:
y_train

In [ ]:
x_train, x_valid, x_test, y_train, y_valid, y_test = prepare_ml_dataset.prepare_ml_dataset(energy, species)

print(x_train.shape, x_valid.shape, x_test.shape, y_train.shape, y_valid.shape, y_test.shape)

In [ ]:
para_name = "learning_rate"
para_set = [1.e-4, 1.5e-3, 1.e-3]

final_train_loss = np.zeros(len(para_set))
final_valid_loss = np.zeros(len(para_set))
total_history = dict()
valid_r2s = np.zeros(len(para_set))

for ipara in range(len(para_set)):
    parameter = para_set[ipara]

    model, history, valid_r2 = train_nn_model.nn_model(x_train, y_train, x_valid, y_valid, data_settings["y_name_log"], 
                                                       output_dir = directories["training_output_dir"] , n_neurons = 18, patience = 32,
                                                       learning_rate = parameter, epochs = 10, batch_size = 8)
    
    total_history[str(parameter)] = history.history
    final_train_loss[ipara] = history.history['loss'][-1]
    final_valid_loss[ipara] = history.history['val_loss'][-1]
    valid_r2s[ipara] = valid_r2